# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [195]:
from smartcard import ATR
from smartcard.System import readers
from smartcard.CardConnection import CardConnection

In [196]:
rarr=readers()
if len(rarr) == 0:
    print("No readers detected")
else:
    reader = rarr[0]
    print(reader)

Generic Smart Card Reader Interface


## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [197]:
# We know that j3h145-jcop3 card supports T1 protocol, so we use that ignoring ATR
protocol = CardConnection.T1_protocol
connection = reader.createConnection()
connection.connect(protocol)

In [198]:
atr=bytes(connection.getATR())
atr.hex()

'3bdc18ff8191fe1fc38073c821136605036351000250'

## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [199]:
def select(appletID):
    """
    Selects an applet by AppletID (should be hex-encoded string).
    Throws an error with the error code if it failed selecting the applet.
    Useful link for error codes: https://www.eftlab.com/knowledge-base/complete-list-of-apdu-responses/
    """
    data = list(bytes.fromhex(appletID))
    # Select:
    # CLA = 0x00
    # INS = 0xA4
    # P1 = 0x04
    # P2 = 0x00
    # Data = the instance AID
    cmd = [0x00, # CLA
              0xA4, # INS
              0x04, # P1
              0x00, # P2
              len(data), # Lc (content length)
          ] + data + [0x00]
    print("Sent command:", bytes(cmd).hex().upper())
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [200]:
# MemoryCardApplet has AID of `B00B5111CB01`
select("B00B5111CB01")

Sent command: 00A4040006B00B5111CB0100


b''

In [201]:
def request(APDU):
    """
    Sends APDU command to the card and returns data.
    APDU should be hex-encoded.
    Throws an error with the error code if it failed selecting the applet.
    Useful link for error codes: https://www.eftlab.com/knowledge-base/complete-list-of-apdu-responses/
    """
    cmd = list(bytes.fromhex(APDU))
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

# 2. Establish secure communication

For secure communication we use [noise protocol](https://noiseprotocol.org/noise.html) with:
- secp256k1 as signature scheme
- AES CBC with random IV as a symmetric cipher
- SHA256 as a hash function, HMAC_SHA256 as authentication code

Secure communication starts with the handshake:
- host generates an ephemeral key pair (random) and sends pubkey to the card
- card generates an ephemeral key pair and responces with ephemeral pubkey and encrypted static pubkey (using DH(e,e) as a secret)

This part is required to establish a shared secret between the card and the host.<br>
Because we want encrypted and authenticated communication.<br>

For key agreement we use ECDH - get card's pubkey, multiply by our secret, use x-coord of the resulting point as shared secret. Maybe sha256 of it?

## 2.1. Get public key from the card

In [202]:
# embit library: https://github.com/diybitcoinhardware/embit
# works with python3 and micropython
from embit.ec import PrivateKey, PublicKey, secp256k1

In [203]:
# get pubkey - private key is generated when applet is installed
sec = request("B0A1000000")
card_pubkey = PublicKey.parse(sec)
card_pubkey

PublicKey(0423ce0eab9ec07806d16174d9496f1e02470a4a3abbc57c1961f3ab3aff7e08f08dd4cb8139eccf3349b5c82930cae129b6800937bbc83dd054416fddf56388c0)

## 2.2. Select a random secret on the host

In [204]:
import os
import hashlib
secret = os.urandom(32)
# ugly copy...
pub = PublicKey.parse(card_pubkey.sec())
secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
shared_secret = pub.sec()[1:33]
shared_hash = hashlib.sha256(shared_secret).digest()
print("Shared secret:", shared_secret.hex())
print("Sha256 of the secret:", shared_hash.hex())

Shared secret: 4a70fe653300a7f9ff06ed6ed916f680c54431f46d8a05b02b6c80fec0a2346e
Sha256 of the secret: ac00f324c7c8159aa66195c74063f7140402a8a1dd2b0437a4612caa2493c314


In [205]:
host_prv = PrivateKey(secret)
host_pub = host_prv.get_public_key()
host_pub.compressed = False
data = bytes([65])+host_pub.sec()
data.hex()

'410499d16589ce59a86f6fa89508056b1b38ae4f108301879b5db0b31b28f56b046798e3307c6d76e0838d5c7e4b050249b2fbbd0481899035e277cd339904ce21ba'

In [206]:
res = request("B0A30000"+data.hex())
if res == shared_hash:
    print("Yey! We have a shared secret now!")
else:
    print("Meh... Something didn't work? Card returned %s, we have %s" % (res.hex(), shared_hash.hex()))

Yey! We have a shared secret now!


In [207]:
# get random data
rand = request("B0A2000000")
rand.hex()

'18be2f8b6abcf7470059822b7e0fe45c316be36422367882c450f190cf6cc185'

In [208]:
# static data for any other APDU
data = request("B000000000")
data

b'Memory cards are not safu so what?'

In [209]:
(bytes([len(data)])+data).hex()

'224d656d6f727920636172647320617265206e6f74207361667520736f20776861743f'

In [213]:
# static data for any other APDU
c = request("B0A40000"+(bytes([len(data)])+data).hex())
c

b"\xf9W\xdc\x0b&\x7f\x05\xe0\xa3\t\x19\xe3\xb74\xdaW\xac\xf3\xf8\xfa\xf5\x18\xd72\xa7\x8b\x1bq\xf3^%\xd3\x1d\xf9[\x02\xd4\x8ae\xb9'\x02\xe6Zu\xb1\xdc\xb9(\x96\xefU/\xf3\xbc\xdb\x86\xe6\x05\x848x\x9b\xd0\xf1G\xf0\xf94\xbf\x92\xf3*\xd4\xda5\xec\x15<<\x8a\xca\xc7\n2J\xf6\x95l\xefh8\x87`mX"

In [214]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import hmac

def decrypt(c,secret):
    recv_hmac = c[-32:]
    iv = c[:16]
    ct = c[16:-32]
    h = hmac.new(shared_secret, digestmod='sha256')
    h.update(iv)
    h.update(ct)
    expected_hmac = h.digest()
    if expected_hmac != recv_hmac:
        raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
    backend = default_backend()
    cipher = Cipher(algorithms.AES(secret), modes.CBC(iv), backend=backend)
    decryptor = cipher.decryptor()
    # remove \x80... padding
    plain = decryptor.update(ct)+decryptor.finalize()
    return b"".join(plain.split(b"\x80")[:-1])

decrypt(c, shared_secret)

b'Memory cards are not safu so what?'

In [215]:
connection.disconnect()